In [1]:
import pandas as pd
import os
import json
import xmltodict
import json
import xml.etree.ElementTree as ET



In [24]:
# parse nodes - generic parser

def parse_dtsx(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    # Define namespaces
    namespaces = {
        'DTS': 'www.microsoft.com/SqlServer/Dts'
    }
    
    # Function to parse Executable elements
    def parse_executables(executables):
        for executable in executables:
            exec_type = executable.attrib.get(f'{{{namespaces["DTS"]}}}ExecutableType')
            name = executable.attrib.get(f'{{{namespaces["DTS"]}}}Name')
            refid = executable.attrib.get(f'{{{namespaces["DTS"]}}}refId')
            description = executable.attrib.get(f'{{{namespaces["DTS"]}}}Description')
            object_data = executable.attrib.get(f'{{{namespaces["DTS"]}}}ObjectData')


            print(f'Executable: {name}, Type: {exec_type}, refId: {refid}, description: {description}, object data: {object_data}')
            
            # Check for nested executables
            nested_executables = executable.findall('DTS:Executables/DTS:Executable', namespaces)
            if nested_executables:
                parse_executables(nested_executables)


    
    # Parse control flow
    print("Control Flow:")
    executables = root.findall('.//DTS:Executable', namespaces)
    parse_executables(executables)
    
    # Parse data flow components
    print("\nData Flow Components:")
    data_flows = root.findall('.//DTS:Executable[@DTS:ExecutableType="SSIS.Pipeline.2"]/DTS:ObjectData/pipeline/components/component', namespaces)
    for component in data_flows:
        comp_name = component.attrib.get('name')
        comp_class_id = component.attrib.get('componentClassID')
        print(f'Component: {comp_name}, ClassID: {comp_class_id}')

# Example usage
file_path = 'data/DailyETLMain.dtsx'
parse_dtsx(file_path)


Control Flow:
Executable: None, Type: Microsoft.ExpressionTask, refId: Package\Calculate ETL Cutoff Time backup, description: Expression Task, object data: None
Executable: None, Type: Microsoft.ExecuteSQLTask, refId: Package\Ensure Date Dimension includes current year, description: Execute SQL Task, object data: None
Executable: None, Type: STOCK:SEQUENCE, refId: Package\Load City Dimension, description: Sequence Container, object data: None
Executable: None, Type: Microsoft.Pipeline, refId: Package\Load City Dimension\Extract Updated City Data to Staging, description: Data Flow Task, object data: None
Executable: None, Type: Microsoft.ExecuteSQLTask, refId: Package\Load City Dimension\Get Last City ETL Cutoff Time, description: Execute SQL Task, object data: None
Executable: None, Type: Microsoft.ExecuteSQLTask, refId: Package\Load City Dimension\Get Lineage Key, description: Execute SQL Task, object data: None
Executable: None, Type: Microsoft.ExecuteSQLTask, refId: Package\Load Cit

In [1]:
# parse nodes - DailyETLMain.dtsx specific


# Define namespaces
namespaces = {
    'DTS': 'www.microsoft.com/SqlServer/Dts'
}

# Function to parse Executable elements (Control Flow)
def parse_executables(executables, depth=0):
    executables_info = []
    for executable in executables:
        exec_type = executable.attrib.get(f'{{{namespaces["DTS"]}}}ExecutableType')
        name = executable.attrib.get(f'{{{namespaces["DTS"]}}}Name')
        refid = executable.attrib.get(f'{{{namespaces["DTS"]}}}refId')
        description = executable.attrib.get(f'{{{namespaces["DTS"]}}}Description')
        object_data = executable.attrib.get(f'{{{namespaces["DTS"]}}}ObjectData')

        executables_info.append({'name': name, 'type': exec_type, 'depth': depth, 'refid': refid, 'description': description, 'object_data': object_data})
        
        # Check for nested executables
        nested_executables = executable.findall('DTS:Executables/DTS:Executable', namespaces)
        if nested_executables:
            executables_info.extend(parse_executables(nested_executables, depth + 1))
    
    return executables_info

# Function to parse Data Flow components
def parse_data_flows(data_flow_tasks):
    data_flow_info = []
    for task in data_flow_tasks:
        task_name = task.attrib.get(f'{{{namespaces["DTS"]}}}Name')
        components = task.findall('.//component', namespaces)
        for component in components:
            comp_name = component.attrib.get('name')
            comp_class_id = component.attrib.get('componentClassID')
            data_flow_info.append({'task_name': task_name, 'component_name': comp_name, 'classID': comp_class_id})
    return data_flow_info

# Load and parse the uploaded .dtsx file
def parse_dtsx(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Parse control flow
    control_flow = parse_executables(root.findall('.//DTS:Executable', namespaces))
    
    # Parse data flow tasks
    data_flow_tasks = root.findall('.//DTS:Executable[@DTS:ExecutableType="Microsoft.Pipeline"]', namespaces)
    data_flow_info = parse_data_flows(data_flow_tasks)
    
    return control_flow, data_flow_info

# Example usage
file_path = 'data/DailyETLMain.dtsx'
control_flow, data_flow_info = parse_dtsx(file_path)

(control_flow, data_flow_info)


([{'name': None,
   'type': 'Microsoft.ExpressionTask',
   'depth': 0,
   'refid': 'Package\\Calculate ETL Cutoff Time backup',
   'description': 'Expression Task',
   'object_data': None},
  {'name': None,
   'type': 'Microsoft.ExecuteSQLTask',
   'depth': 0,
   'refid': 'Package\\Ensure Date Dimension includes current year',
   'description': 'Execute SQL Task',
   'object_data': None},
  {'name': None,
   'type': 'STOCK:SEQUENCE',
   'depth': 0,
   'refid': 'Package\\Load City Dimension',
   'description': 'Sequence Container',
   'object_data': None},
  {'name': None,
   'type': 'Microsoft.Pipeline',
   'depth': 1,
   'refid': 'Package\\Load City Dimension\\Extract Updated City Data to Staging',
   'description': 'Data Flow Task',
   'object_data': None},
  {'name': None,
   'type': 'Microsoft.ExecuteSQLTask',
   'depth': 1,
   'refid': 'Package\\Load City Dimension\\Get Last City ETL Cutoff Time',
   'description': 'Execute SQL Task',
   'object_data': None},
  {'name': None,
   '

In [2]:
# different approach: convert to json (json is easier to read)

def convert_xml_to_json(xml_file, json_file):
    with open(xml_file, 'r') as file:
        xml_content = file.read()
    
    # Convert XML to a dictionary
    data_dict = xmltodict.parse(xml_content)
    
    # Convert dictionary to JSON
    json_data = json.dumps(data_dict, indent=4)
    
    # Write JSON data to a file
    with open(json_file, 'w') as file:
        file.write(json_data)
    
    print(f"Converted {xml_file} to {json_file}")

# Specify the input XML file and output JSON file
xml_file = 'data/Lesson 6.dtsx'
json_file = 'data/output.json'

convert_xml_to_json(xml_file, json_file)


Converted data/Lesson 6.dtsx to output.json


In [18]:
class Load():
    def __init__(self, path):
        self.path = path


    def remove_at_signs(self, obj):
        if isinstance(obj, dict):
            return {key.replace('@', ''): self.remove_at_signs(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self.remove_at_signs(item) for item in obj]
        else:
            return obj
        

    def remove_first_layer(self, json_dict):
        # Extract values from the first layer
        values = list(json_dict.values())
        return values

    def run(self):

        # Path to the XML file
        with open(self.path, 'rb') as f:
            self.xml = f.read()

   
        # open the xml file
        o = xmltodict.parse(self.xml)  # every time you reload the file in colab the key changes (file (1).xml becomes file (2).xml ...)

        json = self.remove_first_layer(self.remove_at_signs(o))[0]

        return json
    




files = [file for file in os.listdir('data/')]
#files = [file for file in os.listdir('data/Creating a Simple ETL Package/Completed Packages/')]

nodes_list = []
filter_list = []

for file in files[:3]:
    

    if file.endswith('.dtsx'):
        print(file)
        json = Load(f"data/{file}").run()
        #json = Load(f'data/Creating a Simple ETL Package/Completed Packages/{file}').run()
    #print(json)



DailyETLMain.dtsx


In [19]:
json

{'xmlns:DTS': 'www.microsoft.com/SqlServer/Dts',
 'DTS:refId': 'Package',
 'DTS:CreationDate': '4/10/2016 11:13:19 AM',
 'DTS:CreationName': 'Microsoft.Package',
 'DTS:CreatorComputerName': 'GREG7440WIN10',
 'DTS:CreatorName': 'GREG7440WIN10\\Greg',
 'DTS:DTSID': '{1688CC04-248C-44DC-9B43-C8D2DE1DE3FD}',
 'DTS:ExecutableType': 'Microsoft.Package',
 'DTS:LastModifiedProductVersion': '13.0.4001.0',
 'DTS:LocaleID': '3081',
 'DTS:ObjectName': 'DailyETLMain',
 'DTS:PackageType': '5',
 'DTS:ProtectionLevel': '0',
 'DTS:VersionBuild': '62',
 'DTS:VersionGUID': '{00A1E197-21DE-4954-AAA0-E348461E172E}',
 'DTS:Property': {'DTS:Name': 'PackageFormatVersion', '#text': '8'},
 'DTS:Variables': {'DTS:Variable': [{'DTS:CreationName': '',
    'DTS:DTSID': '{60BD9010-4784-49A6-8FF2-340D7AB3B70D}',
    'DTS:IncludeInDebugDump': '6789',
    'DTS:Namespace': 'User',
    'DTS:ObjectName': 'LastETLCutoffTime',
    'DTS:VariableValue': {'DTS:DataType': '7',
     '#text': '4/11/2016 10:39:10 AM'}},
   {'DTS:C

In [17]:
# for package.dtsx

print(json['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component'][0]['name'])
print(json['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component'][1]['name'])
print(json['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component'][2]['name'])

Extract Sample Currency Data
Lookup Currency Key
Lookup Date Key


In [ ]:
# control flow nodes

for node in json['DTS:Executables']['DTS:Executable']:#['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component'][0]['name'])
    print(node['DTS:ObjectName'])
    print(node['DTS:Description'])
    print()

Calculate ETL Cutoff Time backup
Expression Task

Ensure Date Dimension includes current year
Execute SQL Task

Load City Dimension
Sequence Container

Load Customer Dimension
Sequence Container

Load Employee Dimension
Sequence Container

Load Movement Fact
Sequence Container

Load Order Fact
Sequence Container

Load Payment Method Dimension
Sequence Container

Load Purchase Fact
Sequence Container

Load Sale Fact
Sequence Container

Load Stock Holding Fact
Sequence Container

Load Stock Item Dimension
Sequence Container

Load Supplier Dimension
Sequence Container

Load Transaction Fact
Sequence Container

Load Transaction Type Dimension
Sequence Container

Trim Any Milliseconds
Expression Task



In [ ]:
# control nodes and data flow nodes

# control node for loop
for control_node in json['DTS:Executables']['DTS:Executable']:
    print(control_node)
    print(" control node name: ",control_node['DTS:ObjectName'])
    print()

    if control_node['DTS:Description'] == 'Sequence Container':
        for executable in control_node['DTS:Executables']['DTS:Executable']:
            print("     ", executable)
            print("     executable name: ", executable['DTS:ObjectName'])
            print()

            if executable['DTS:CreationName'] == 'Microsoft.Pipeline':

                for component in executable['DTS:ObjectData']['pipeline']['components']['component']:
                    print("          ", component)
                    print("          componenet name: ", component['name'])
                    print()
            elif executable['DTS:CreationName'] =='Microsoft.ExecuteSQLTask':
                print("      ", executable['DTS:ObjectData']['SQLTask:SqlTaskData']['SQLTask:SqlStatementSource'])
                print()

            else:
                print(executable['DTS:CreationName'])
                print(executable)

                print()

      

 #           try:
 #               print('Output columns:')
 #               if type(node['outputs']['output'][0]['outputColumns']['outputColumn']) == list:
 #                   for col in node['outputs']['output'][0]['outputColumns']['outputColumn']:
 #                       print(col['name'])
 #               else:
 #                   print(node['outputs']['output'][0]['outputColumns']['outputColumn']['name'])
 #           except:
 #               pass
 #           try:
 #               print('Input columns:')
#
 #               if type(node['inputs']['input']['inputColumns']['inputColumn']) == list:
 #                   for col in node['inputs']['input']['inputColumns']['inputColumn']:
 #                       print(col['cachedName'])
 #               else:
 #                   print(node['inputs']['input']['inputColumns']['inputColumn']['name'])
 #           except:
 #               pass
 #           print()
 #       print()
 #       print("######################")
 #       print()

{'DTS:refId': 'Package\\Calculate ETL Cutoff Time backup', 'DTS:CreationName': 'Microsoft.ExpressionTask', 'DTS:Description': 'Expression Task', 'DTS:DTSID': '{1AB43A83-F682-4D1A-B54D-5B6D71A27FBD}', 'DTS:ExecutableType': 'Microsoft.ExpressionTask', 'DTS:LocaleID': '-1', 'DTS:ObjectName': 'Calculate ETL Cutoff Time backup', 'DTS:TaskContact': 'Expression Task;Microsoft Corporation; SQL Server 2016 RC2; © 2015 Microsoft Corporation; All Rights Reserved;http://www.microsoft.com/sql/support/default.asp;1', 'DTS:ThreadHint': '0', 'DTS:Variables': None, 'DTS:ObjectData': {'ExpressionTask': {'Expression': '@[User::TargetETLCutoffTime] = DATEADD("Minute", -5, GETUTCDATE()  )'}}}
 control node name:  Calculate ETL Cutoff Time backup

{'DTS:refId': 'Package\\Ensure Date Dimension includes current year', 'DTS:CreationName': 'Microsoft.ExecuteSQLTask', 'DTS:Description': 'Execute SQL Task', 'DTS:DTSID': '{92E7FED7-B5B8-4831-8623-7E5C7931DDF8}', 'DTS:ExecutableType': 'Microsoft.ExecuteSQLTask', 'D

In [ ]:
control_node

{'DTS:refId': 'Package\\Load City Dimension',
 'DTS:CreationName': 'STOCK:SEQUENCE',
 'DTS:Description': 'Sequence Container',
 'DTS:DTSID': '{035B143D-3ADA-42F4-8824-972E2DE9514C}',
 'DTS:ExecutableType': 'STOCK:SEQUENCE',
 'DTS:LocaleID': '-1',
 'DTS:ObjectName': 'Load City Dimension',
 'DTS:Variables': None,
 'DTS:Executables': {'DTS:Executable': [{'DTS:refId': 'Package\\Load City Dimension\\Extract Updated City Data to Staging',
    'DTS:CreationName': 'Microsoft.Pipeline',
    'DTS:Description': 'Data Flow Task',
    'DTS:DTSID': '{F2DAF031-DD28-49EA-95CA-E6E97A3194CE}',
    'DTS:ExecutableType': 'Microsoft.Pipeline',
    'DTS:LocaleID': '-1',
    'DTS:ObjectName': 'Extract Updated City Data to Staging',
    'DTS:TaskContact': 'Performs high-performance data extraction, transformation and loading;Microsoft Corporation; Microsoft SQL Server; (C) Microsoft Corporation; All Rights Reserved;http://www.microsoft.com/sql/support/default.asp;1',
    'DTS:Variables': None,
    'DTS:Object

In [ ]:
# data flow nodes (with sql statements)

for node in json['DTS:Executables']['DTS:Executable']['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component']:
    print("Name node:", node['name'])
    print()
    print("Description node:", node['description'])
    print()

    try:
        print('Output columns:')
        if type(node['outputs']['output'][0]['outputColumns']['outputColumn']) == list:
            for col in node['outputs']['output'][0]['outputColumns']['outputColumn']:
                print(col['name'])
        else:
            print(node['outputs']['output'][0]['outputColumns']['outputColumn']['name'])
    except:
        pass
    try:
        print('Input columns:')

        if type(node['inputs']['input']['inputColumns']['inputColumn']) == list:
            for col in node['inputs']['input']['inputColumns']['inputColumn']:
                print(col['cachedName'])
        else:
            print(node['inputs']['input']['inputColumns']['inputColumn']['name'])
    except:
        pass
    print()
    print("######################")
    print()
        


KeyError: 'DTS:Executables'

In [ ]:
json['DTS:Executables']['DTS:Executable']['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component'][3]


{'refId': 'Package\\Foreach File in Folder\\Extract Sample Currency Data\\Lookup Currency Key',
 'name': 'Lookup Currency Key',
 'componentClassID': '{671046B0-AA63-4C9F-90E4-C06E0B710CE3}',
 'description': "Joins additional columns to the data flow by looking up values in a table. For example, join to the 'employee id' column the employees table to get 'hire date' and 'employee name'. We recommend this transformation when the lookup table can fit into memory.",
 'usesDispositions': 'true',
 'version': '6',
 'contactInfo': 'Lookup;Microsoft Corporation; Microsoft SQL Server; (C) Microsoft Corporation; All Rights Reserved; http://www.microsoft.com/sql/support;6',
 'properties': {'property': [{'name': 'SqlCommand',
    'dataType': 'System.String',
    'description': 'Specifies the SQL statement that generates the lookup table.',
    'UITypeEditor': 'Microsoft.DataTransformationServices.Controls.ModalMultilineStringEditor, Microsoft.DataTransformationServices.Controls, Version=11.0.0.0, C

In [ ]:
# locate name source tables and transformation
# extract sql code from each node

#for property in json['DTS:Executables']['DTS:Executable']['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component'][4]['properties']['property']:
#    print(property['#text'])

for node in json['DTS:Executables']['DTS:Executable']['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component']:
    print("Name node:", node['name'])
    print()
    try:

        print("Connection db:", node['connections']['connection']['connectionManagerID'])
    except:
        pass
    print()

    # db: connections connection database connectionManagerId

    for property in node['properties']['property']:
        if 'sql' in property['name'].lower():
            if '#text' in property.keys():
                print(property['#text'])
                print()
    print('#########################')
    print()




KeyError: 'DTS:Executables'

In [ ]:
json['DTS:Executables']['DTS:Executable']['DTS:Executables']['DTS:Executable']['DTS:ObjectData']['pipeline']['components']['component'][4]['connections']['connection']['connectionManagerID']

KeyError: 'DTS:Executables'